# [신경망 모델 구성하기](https://tutorials.pytorch.kr/beginner/basics/buildmodel_tutorial.html)

신경망은 데이터에 대한 연산을 수행하는 계층(layer)/모듈(module)로 구성되어 있다.

<a href="https://pytorch.org/docs/stable/nn.html">`torch.nn`</a> 네임스페이스는 신경망을 구성하는데 필요한 모든 구성 요소를 제공한다.

PyTorch의 모든 모듈은 <a href="https://pytorch.org/docs/stable/generated/torch.nn.Module.html">`nn.Module`</a>의 하위 클래스(subclass) 이다.

신경만은 다른 모듈(계층: layer)로 구성된 모듈이다. 이러한 중첩된 구조는 복잡한 아키텍처를 쉽게 구축하고 관리할 수 있다.

FashionMNIST 데이터셋의 이미지들을 분류하는 신경망을 구성해보도록 하자.

In [2]:
# 라이브러리 불러오기
import os

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## 학습을 위한 장치 얻기

가능한 경우 GPU와 같은 하드웨어 가속기에서 모델을 학습하는 것이 좋다.

`torch.cuda`를 사용할 수 있는지 확인하고 그렇지 않으면 CPU를 계속 사용하도록 하자.

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Using {device} device")

Using cpu device


## 클래스 정의하기

신경망 모델을 `nn.Module`의 하위클래스로 정의하고, `__init__` 에서 신경망 계층들을 초기화한다.

`nn.Module`을 상속받은 모든 클래스는 `forward` 메소드에 입력 데이터에 대한 연산들을 구현한다.

In [4]:
class NeuralNetwork(nn.Module):
    
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(in_features = 28 * 28, out_features = 512),
            nn.ReLU(),
            nn.Linear(in_features = 512, out_features = 512),
            nn.ReLU(),
            nn.Linear(512, 10))
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        
        return logits

`NeuralNetwork`의 인스턴스(instance)를 생성하고 이를 `device`로 이동한 뒤, 구조(structure)를 출력한다.

- 위의 `NeuralNetwork` 클래스는 `nn.Module` 클래스를 상속받고 있음을 확인할 수 있다.

```python
class NeuralNetwork(nn.Module):
    
    def __init__(self):
        super(NeuralNetwork, self).__init__()
```

- `super` 명령어는 상속관계에서 상속의 대상인 부모 클래스를 호출하는 함수이다.

- `super()` 의 인자로는 두 개가 전달되며, **하위클래스의 이름**과 **하위클래스의 객체**가 필요하다.

In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


모델을 사용하기 위해 입력 데이터를 전달한다. 이는 일부 백그라운드 연산들과 함께 모델의 `forward`를 실행한다.

이 때, `model.forward()` 를 직접 호출하지 말 것!

모델에 입력을 호출하면 각 분류(class)에 대한 원시(raw) 예측값이 있는 10-차원 텐서가 반환된다.

원시 예측값을 `nn.Softmax` 모듈의 인스턴스에 통과시켜 예측 확률을 얻는다.

In [14]:
# 하나의 임의 입력 데이터 생성
X = torch.rand(1, 28, 28, device = device)

print("Rank of tensor :", X.dim(), "\n")
print("Shape of tensor :", X.shape)

Rank of tensor : 3 

Shape of tensor : torch.Size([1, 28, 28])


In [26]:
# 모델에 입력을 호출하면 각 분류에 대한 예측값을 반환한다.
logits = model(X)

print("Shape of logits tensor :", logits.size(), "\n")
print(logits)

Shape of logits tensor : torch.Size([1, 10]) 

tensor([[ 0.0014, -0.1477,  0.0006,  0.0229, -0.1643,  0.0897,  0.0015,  0.0089,
         -0.0832, -0.0145]], grad_fn=<AddmmBackward0>)


In [29]:
# nn.Softmax 모듈을 사용하여 예측 확률 반환
pred_probab = nn.Softmax(dim = 1)(logits)

print("Shape of softmax tensor :", pred_probab.size(), "\n")
print(pred_probab)

Shape of softmax tensor : torch.Size([1, 10]) 

tensor([[0.1027, 0.0885, 0.1027, 0.1050, 0.0871, 0.1122, 0.1028, 0.1035, 0.0944,
         0.1011]], grad_fn=<SoftmaxBackward0>)


In [35]:
# 예측 확률 중 가장 큰 값을 추출
y_pred = pred_probab.argmax(dim = 1)
print(f"Predicted class : {y_pred}")

Predicted class : tensor([5])


---

## 모델 계층(Layer)

FashionMNIST 모델의 계층들을 살펴보도록 하자.

이를 설명하기 위해, 28x28 크기의 이미지 3개로 구성된 미니배치를 가져와, 신경망을 통과할 때 어떤 일이 발생하는지 알아보자.

In [37]:
input_image = torch.rand(3, 28, 28)
print("Shape of tensor :", input_image.size())

Shape of tensor : torch.Size([3, 28, 28])


### nn.Flatten

신경망 모델에 이미지 데이터를 입력해주기 위해서는 2D 이미지를 평탄화 해주는 작업을 수행해줘야 한다.

평탄화 작업을 수행해주는 모듈이 `nn.Flatten` 이다.

<a href="https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html">`nn.Flatten`</a> 계층을 초기화하여 각 28x28의 2D 이미지를 784 픽셀 값을 갖는 연속된 배열로 변환한다.

In [38]:
# nn.Flatten 객체 생성
flatten = nn.Flatten()

# 이미지 평탄화 작업 수행
flat_image = flatten(input_image)
print("Shape of flatten image tensor:", flat_image.size())

Shape of flatten image tensor: torch.Size([3, 784])


### nn.Linear

<a href="https://pytorch.org/docs/stable/generated/torch.nn.Linear.html">`nn.Linear`</a> 계층은 저장된 가중치(weight)와 편향(bias)을 사용하여 입력에 선형 변환(linear transformation)을 적용하는 모듈이다.

In [39]:
# nn.Linear 객체 생성
layer1 = nn.Linear(in_features = 28 * 28, out_features = 20)

# 생성한 객체에 데이터 입력
hidden1 = layer1(flat_image)
print("Shape of linear image tensor :", hidden1.size())

Shape of linear image tensor : torch.Size([3, 20])


### nn.ReLU

비선형 활성화(activation)는 모델의 입력과 출력 사이에 복잡한 관계(mapping)을 만든다.

비선형 활성화는 선형 변환 후에 적용되어 비선형성(nonlinearity)을 도입하고, 신경망이 다양한 형상을 학습할 수 있도록 도와준다.

이 모델에서는 <a href=https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html>`nn.ReLU`</a>를 선형 계층들 사이에 사용하지만, 모델을 만들 때는 비선형성을 가진 다른 활성화를 도입할 수도 있다.

In [46]:
print(f"Before ReLU \n\n {hidden1} \n\n")

hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU \n\n {hidden1}")

Before ReLU 

 tensor([[ 0.2497, -0.6188, -0.6274,  0.0762,  0.1060,  0.4084, -0.5819, -0.0975,
         -0.1819, -0.2293, -0.2323, -0.0100, -0.1505, -0.0419, -0.0428,  0.1659,
         -0.4777,  0.1197,  0.4844, -0.3187],
        [-0.0048, -0.5053, -0.5270,  0.0321,  0.2664,  0.2884, -0.2082, -0.1869,
         -0.2774, -0.3041, -0.1134,  0.1100, -0.2214, -0.2591,  0.0032,  0.1003,
         -0.2098, -0.0208,  0.2437, -0.1151],
        [-0.1704, -0.4867, -0.4355,  0.0846, -0.1359,  0.4445, -0.0812, -0.2342,
         -0.2314, -0.2918, -0.3480, -0.2383,  0.0238,  0.1212,  0.3451, -0.0300,
         -0.2576, -0.0380,  0.3660, -0.0582]], grad_fn=<AddmmBackward0>) 


After ReLU 

 tensor([[0.2497, 0.0000, 0.0000, 0.0762, 0.1060, 0.4084, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1659, 0.0000, 0.1197,
         0.4844, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0321, 0.2664, 0.2884, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.1100, 0.0000, 0.0000,

### nn.Sequential

<a href="https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html">`nn.Sequential`</a>은 순서를 갖는 모듈의 컨테이너이다. 데이터는 정의된 것과 같은 순서로 모든 모듈들을 통해 전달된다.

순차 컨테이너(sequential container)를 사용하면 아래의 `seq_modules`와 같은 신경망을 빠르게 만들 수 있다.

In [49]:
# 신경망 모형
seq_modules = nn.Sequential(
    nn.Flatten(),
    nn.Linear(in_features = 28 * 28, out_features = 20),
    nn.ReLU(),
    nn.Linear(in_features = 20, out_features = 10))

input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

print("Shape of logits tensor :", logits.size())

Shape of logits tensor : torch.Size([3, 10])


### nn.Softmax

신경망의 마지막 선형 계층은 <a href="https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html">`nn.Softmax`</a> 모듈에 전달될 ([-∞, ∞] 범위의 원시 값(raw value)인) logits를 반환한다.

logits는 모델의 각 분류(class)에 대한 예측 확률을 나타내도록 [0, 1] 범위로 비례하여 조정(scale) 된다.

In [50]:
softmax = nn.Softmax(dim = 1)

pred_probab = softmax(logits)
print(pred_probab)

tensor([[0.1217, 0.1011, 0.1035, 0.1102, 0.1162, 0.0822, 0.0738, 0.0750, 0.1314,
         0.0848],
        [0.1199, 0.0987, 0.1115, 0.1353, 0.1160, 0.0708, 0.0735, 0.0728, 0.1178,
         0.0836],
        [0.1182, 0.0988, 0.1075, 0.1284, 0.1026, 0.0829, 0.0655, 0.0853, 0.1242,
         0.0866]], grad_fn=<SoftmaxBackward0>)


## 모델 매개변수

신경망 내부의 많은 계층들은 **매개변수화(parameterize)** 된다. 즉, 학습 중에 최적화되는 가중치와 편향과 연관지어진다.

`nn.Module`을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 추적(track)되며, 모델의 `parameters()` 및 `named_parameters()` 메소드로 모든 매개변수에 접근할 수 있게 된다.

이 예제에서는 각 매개변수들을 순회하며, 매개변수의 크기와 값을 출력해보도록 하자.

In [60]:
print("Model structure : \n\n", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer : {name} \n")
    print(f"Size : {param.size()} \n")
    print(f"Values : {param[:2]} \n\n")

Model structure : 

 NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
) 


Layer : linear_relu_stack.0.weight 

Size : torch.Size([512, 784]) 

Values : tensor([[ 0.0008, -0.0091,  0.0331,  ..., -0.0275,  0.0231,  0.0146],
        [-0.0127,  0.0049, -0.0117,  ...,  0.0099, -0.0144,  0.0353]],
       grad_fn=<SliceBackward0>) 


Layer : linear_relu_stack.0.bias 

Size : torch.Size([512]) 

Values : tensor([-0.0066, -0.0348], grad_fn=<SliceBackward0>) 


Layer : linear_relu_stack.2.weight 

Size : torch.Size([512, 512]) 

Values : tensor([[ 0.0200, -0.0050, -0.0217,  ..., -0.0218, -0.0185, -0.0175],
        [-0.0006,  0.0027, -0.0371,  ...,  0.0095,  0.0163, -0.0311]],
       grad_fn=<SliceBackward0>) 


Layer : linear_relu_